In [117]:
import torch
import numpy as np
import imageio.v2 as imageio

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_dtype(torch.float32)

In [118]:
trans_t = lambda t: torch.Tensor([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, t],
    [0, 0, 0, 1]]).float()

rot_phi = lambda phi: torch.Tensor([
    [1, 0, 0, 0],
    [0, np.cos(phi), -np.sin(phi), 0],
    [0, np.sin(phi), np.cos(phi), 0],
    [0, 0, 0, 1]]).float()

rot_theta = lambda th: torch.Tensor([
    [np.cos(th), 0, -np.sin(th), 0],
    [0, 1, 0, 0],
    [np.sin(th), 0, np.cos(th), 0],
    [0, 0, 0, 1]]).float()


def pose_spherical(theta, phi, radius):
    c2w = trans_t(radius)
    c2w = rot_phi(phi / 180. * np.pi) @ c2w
    c2w = rot_theta(theta / 180. * np.pi) @ c2w
    c2w = torch.Tensor(np.array([[-1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]])) @ c2w
    return c2w

In [119]:
import json
import os

metas = {}
for s in ['train', 'val', 'test']:
    with open(os.path.join("data/nerf_synthetic/lego", 'transforms_{}.json'.format(s)), 'r') as fp:
        metas[s] = json.load(fp)
imgs = []
poses = []
for frame in metas['train']['frames']:
    path = os.path.normpath(os.path.join("data/nerf_synthetic/lego", frame['file_path'] + '.png'))
    imgs.append(imageio.imread(path).astype(np.float32) / 255.)
    poses.append(np.array(frame['transform_matrix']).astype(np.float32))
height, width = imgs[0].shape[:2]
camera_angle_x = float(metas['train']['camera_angle_x'])
focal = .5 * width / np.tan(.5 * camera_angle_x)
near = 2.
far = 6.
render_poses = torch.stack([pose_spherical(angle, -30.0, 4.0) for angle in np.linspace(-180, 180, 40 + 1)[:-1]], 0)
imgs2 = np.concatenate(imgs, 0)
poses2 = np.concatenate(poses, 0)

In [ ]:
class NeRF(torch.nn.Module):
    def __init__(self):
        super(NeRF, self).__init__()